In [1]:
import langchain
from dotenv import load_dotenv
import json
from tqdm import tqdm

load_dotenv()

True

In [2]:
with open("data.json", 'r') as json_file:
    dict_p = json.load(json_file)

In [3]:

# Load the LLM model
chat = langchain.chat_models.ChatOpenAI(temperature=0.2, model_name='gpt-4',)


def get_basic_chat_chain(system_template: str, user_template: str) -> langchain.LLMChain:
    """
    Returns a langchain chain given an user and chat inputs

    Parameters:
        system_template (str): The system template
        user_template (str): The user template

    Returns:
        langchain.LLMChain: The langchain chain
    """

    system_prompt = langchain.prompts.SystemMessagePromptTemplate.from_template(
        system_template)

    user_prompt = langchain.prompts.HumanMessagePromptTemplate.from_template(
        user_template)

    chat_prompt = langchain.prompts.ChatPromptTemplate.from_messages([
        system_prompt,
        user_prompt
    ])

    return langchain.LLMChain(llm=chat, prompt=chat_prompt)



In [4]:
system_promt = """
Your are an assistant and you are creating a knowledge graph from a scientific article the talks about the moon exploration.
You will be given as input a piece of text from the article. 
Your job is to identify the main concepts to use them as nodes in our knowledge graph. 
Then you have to identify the relations between each concept.

You will return the answer as a JSON object with the keys:
    - "nodes" and the value as a list of strings ['concept1','concept2','concept3',...].
    - "relations" and the value as a list of lists [['concept9','concept10','relation1'],['concept9','concept2','relation2']...]
You have to only return the JSON object.
Your answer begins with a left curly bracket and ends with a right curly bracket.
No preface text.

"""
user_prompt = """
## TEXT:
{text}

"""
chain = get_basic_chat_chain(system_promt,user_prompt)

In [5]:
for p in tqdm(dict_p["paragraphs"][12:]):
    text=p["text"]
    response = chain.run(text=text)
    try:
        p["graph"] = json.loads(response)
    except:
        print(response)



100%|██████████| 44/44 [21:50<00:00, 29.79s/it]


In [6]:
with open("data_graph_bis.json","w") as f:
    json.dump(dict_p, f, indent=4)